# Indexación jerárquica

A menudo, es útil ir más allá y almacenar datos de mayor dimensión, es decir, datos indexados por más de una o dos claves. Si bien Pandas proporciona objetos ``Panel`` y ``Panel4D`` que manejan de forma nativa datos tridimensionales y tetradimensionales, un patrón mucho más común en la práctica es hacer uso de la indexación jerárquica (también conocida como indexación múltiple) para incorporar múltiples niveles dentro de un solo índice. De esta manera, los datos de dimensiones superiores se pueden representar de forma compacta dentro de la familiar unidimensional ``Series`` y `bidimensional ``DataFrame``. 

En esta sección, exploraremos la creación directa de objetos ``MultiIndex``, consideraciones al indexar, rebanar y calcular estadísticas a través de datos indexados de forma múltiple, y rutinas útiles para convertir entre representaciones simples e indexadas jerárquicamente de sus datos.

Comenzamos con las importaciones estándar: 


In [1]:
import pandas as pd
import numpy as np

## Una serie indexada multiplicada

Comencemos por considerar cómo podríamos representar datos bidimensionales dentro de una dimensión ``Series``. Para mayor concreción, consideraremos una serie de datos donde cada punto tiene un carácter y clave numérica.

### El mal camino

Suponga que desea realizar un seguimiento de los datos sobre los estados de dos años diferentes. Al usar las herramientas de Pandas que ya hemos cubierto, es posible que tenga la tentación de simplemente usar las tuplas de Python como claves:

In [21]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

Con este esquema de indexación, puede indexar o dividir la serie directamente en función de este índice múltiple:

In [22]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

Pero la conveniencia termina ahí. Por ejemplo, si necesita seleccionar todos los valores de 2010, deberá hacer algunos cambios complicados (y potencialmente lentos) para que esto suceda:

In [23]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

Esto produce el resultado deseado, pero no es tan limpio (o tan eficiente para grandes conjuntos de datos) como la sintaxis de corte que amamos en Pandas.

### La Mejor Manera: Pandas MultiIndex

Afortunadamente, Pandas proporciona una mejor manera. Nuestra indexación basada en tuplas es esencialmente un índice múltiple rudimentario, y los Pandas tipo ``MultiIndex`` nos da el tipo de operaciones que deseamos tener. Podemos crear un índice múltiple a partir de las tuplas de la siguiente manera:

In [24]:
multiindex = pd.MultiIndex.from_tuples(index)
multiindex

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

Si volvemos a indexar nuestra serie con este MultiIndex, vemos la representación jerárquica de los datos:

In [31]:
pop = pop.reindex(multiindex)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Aquí las dos primeras columnas de la ``Series`` La representación muestra los múltiples valores del índice, mientras que la tercera columna muestra los datos. Observe que faltan algunas entradas en la primera columna: en esta representación de múltiples índices, cualquier entrada en blanco indica el mismo valor que la línea que está encima.

Ahora, para acceder a todos los datos cuyo segundo índice es 2010, simplemente podemos usar la notación de división de Pandas:

In [38]:
pop[:,2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

El resultado es una matriz indexada individualmente con solo las claves que nos interesan. Esta sintaxis es mucho más conveniente (¡y la operación es mucho más eficiente!) que la solución casera de indexación múltiple basada en tuplas con la que comenzamos. Ahora analizaremos más a fondo este tipo de operación de indexación en datos indexados jerárquicamente.

### MultiIndex como dimensión extra

Puede notar algo más aquí: fácilmente podríamos haber almacenado los mismos datos usando un simple ``DataFrame`` con índice y etiquetas de columna. De hecho, Pandas está construido con esta equivalencia en mente. El método ``unstack()`` convertirá rápidamente un indexado múltiple``Series`` en un indexado convencional ``DataFrame``:

In [39]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


El método ``stack()`` proporciona la operación opuesta:

In [40]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Al ver esto, es posible que se pregunte por qué nos molestaríamos con la indexación jerárquica. La razón es simple: así como pudimos usar la indexación múltiple para representar datos bidimensionales dentro de una dimensión ``Series``, también podemos usarlo para representar datos de tres o más dimensiones en una ``Series`` o ``DataFrame``. Cada nivel adicional en un índice múltiple representa una dimensión adicional de datos; aprovechar esta propiedad nos da mucha más flexibilidad en los tipos de datos que podemos representar. Concretamente, podríamos querer agregar otra columna de datos demográficos para cada estado en cada año (digamos, población menor de 18 años); con un ``MultiIndex`` esto es tan fácil como agregar otra columna a la DataFrame:

In [42]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

Además, todos los ufuncs y otras funcionalidades discutidas en Operar con datos en Pandas también funcionan con índices jerárquicos. Aquí calculamos la fracción de personas menores de 18 años por año, dados los datos anteriores:

In [45]:
f_u18 = pop_df['under18'] / pop_df['total']
print(f_u18)
f_u18.unstack()

California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64


,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


Esto nos permite manipular y explorar fácil y rápidamente incluso datos de alta dimensión.

## Métodos de creación de índices múltiples

La forma más sencilla de construir un indexado múltiple Series o DataFrame es simplemente pasar una lista de dos o más matrices de índices al constructor. Por ejemplo: 

In [49]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.273099  0.519308
  2  0.884092  0.330233
b 1  0.047495  0.726948
  2  0.468389  0.904764

El trabajo de crear el ``MultiIndex`` se hace en segundo plano.

De manera similar, si pasa un diccionario con tuplas apropiadas como claves, Pandas lo reconocerá automáticamente y usará un ``MultiIndex`` por defecto:

In [46]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

Sin embargo, a veces es útil crear explícitamente un ``MultiIndex``; Veremos un par de estos métodos aquí.

### Constructores de índices múltiples explícitos

Para obtener más flexibilidad en la forma en que se construye el índice, puede usar los constructores de métodos de clase disponibles en el ``pd.MultiIndex``. Por ejemplo, como hicimos antes, puede construir el ``MultiIndex`` de una lista simple de matrices que dan los valores de índice dentro de cada nivel:

In [47]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Puede construirlo a partir de una lista de tuplas que dan los múltiples valores de índice de cada punto:

In [50]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Incluso puede construirlo a partir de un producto cartesiano de índices individuales:

In [51]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Del mismo modo, puede construir el ``MultiIndex`` directamente usando su codificación interna pasando ``levels``(una lista de listas que contienen valores de índice disponibles para cada nivel) y ``labels``(una lista de listas que hacen referencia a estas etiquetas):

In [55]:
# pd.MultiIndex(levels=[['a', 'b'], [1, 2]], labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

Cualquiera de estos objetos se puede pasar como el ``index`` argumento al crear un ``Series`` o ``Dataframe``, o ser pasado a la ``reindex`` método de un existente ``Series`` o ``DataFrame``.

### Nombres de niveles de MultiIndex

A veces es conveniente nombrar los niveles de la ``MultiIndex``. Esto se puede lograr pasando el argumento ``names`` de cualquiera de los anteriores constructores ``MultiIndex``, o configurando el atributo ``names`` del índice después del hecho:

In [ ]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Con conjuntos de datos más complicados, esta puede ser una forma útil de realizar un seguimiento del significado de varios valores de índice.

### Multiíndice para columnas

En un ``DataFrame``, las filas y las columnas son completamente simétricas y, al igual que las filas pueden tener varios niveles de índices, las columnas también pueden tener varios niveles. Considere lo siguiente, que es una maqueta de algunos datos médicos (algo realistas):

In [62]:
# Indices jerarquicos y columnas
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10000
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject         Bob          Guido            Sue      
type             HR  Temp       HR  Temp       HR  Temp
year visit                                             
2013 1       2037.0  38.6   5037.0  37.2   7037.0  35.3
     2      12037.0  35.4  -9963.0  36.2  -1963.0  37.2
2014 1       2037.0  36.6 -23963.0  37.9  -3963.0  36.5
     2       -963.0  38.2  -3963.0  36.9  15037.0  38.1

Aquí vemos dónde la indexación múltiple para filas y columnas puede ser muy útil. Se trata fundamentalmente de datos cuatridimensionales, donde las dimensiones son el sujeto, el tipo de medida, el año y el número de visita. Con esto en su lugar podemos, por ejemplo, indexar la columna de nivel superior por el nombre de la persona y obtener una ``DataFrameque`` contiene solo la información de esa persona:

In [64]:
health_data['Bob']

type             HR  Temp
year visit               
2013 1       2037.0  38.6
     2      12037.0  35.4
2014 1       2037.0  36.6
     2       -963.0  38.2

Para registros complicados que contienen múltiples medidas etiquetadas en múltiples momentos para muchos temas (personas, países, ciudades, etc.), ¡el uso de filas y columnas jerárquicas puede ser extremadamente conveniente!

## Indexación y división de un índice múltiple

Indexación y corte en un ``MultiIndex`` está diseñado para ser intuitivo y ayuda si piensa en los índices como dimensiones adicionales. Primero veremos la indexación multiplicada indexando ``Series``, y luego indexado de forma múltiple ``DataFrames``.


### Multiplicar Series indexadas

Considere la multiplicación indexada de ``Series`` las poblaciones estatales que vimos antes:

In [65]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Podemos acceder a elementos individuales indexando con múltiples términos:

In [66]:
pop['California', 2000]

33871648

El ``MultiIndex`` también admite la indexación parcial o la indexación de solo uno de los niveles del índice. el resultado es otro ``Series``, con los índices de nivel inferior mantenidos:

In [67]:
pop['California']

2000    33871648
2010    37253956
dtype: int64

El rebanado parcial también está disponible, siempre que el ``MultiIndex`` está ordenado.

In [68]:
pop.loc['California':'New York']

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

Con índices ordenados, la indexación parcial se puede realizar en niveles inferiores pasando un segmento vacío en el primer índice:

In [69]:
pop[:, 2000]

California    33871648
New York      18976457
Texas         20851820
dtype: int64

Otros tipos de indexación y selección también funcionan; por ejemplo, selección basada en máscaras booleanas:

In [71]:
pop[pop > 22000000]

California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

La selección basada en la indexación elegante también funciona:

In [72]:
pop[['California', 'Texas']]

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### DataFrames indexados Multiples

Un indexado múltiple ``DataFrame`` se comporta de manera similar. Considere nuestro ``DataFrame`` médico de antes:

In [73]:
health_data

subject         Bob          Guido            Sue      
type             HR  Temp       HR  Temp       HR  Temp
year visit                                             
2013 1       2037.0  38.6   5037.0  37.2   7037.0  35.3
     2      12037.0  35.4  -9963.0  36.2  -1963.0  37.2
2014 1       2037.0  36.6 -23963.0  37.9  -3963.0  36.5
     2       -963.0  38.2  -3963.0  36.9  15037.0  38.1

Recuerde que las columnas son primarias en un ``DataFrame``, y la sintaxis utilizada para índices múltiples ``Series`` se aplica a las columnas. Por ejemplo, podemos recuperar los datos de frecuencia cardiaca de Guido con una sencilla operación:

In [74]:
health_data['Guido', 'HR']

year  visit
2013  1         5037.0
      2        -9963.0
2014  1       -23963.0
      2        -3963.0
Name: (Guido, HR), dtype: float64

Además, al igual que con el caso de índice único, podemos usar el ``loc``, ``iloc``, y ``ix`` indexadores. Por ejemplo:

In [76]:
health_data.iloc[:2, :2]

subject         Bob      
type             HR  Temp
year visit               
2013 1       2037.0  38.6
     2      12037.0  35.4

Estos indexadores proporcionan una vista tipo matriz de los datos bidimensionales subyacentes, pero cada índice individual en ``loc`` o ``iloc``se puede pasar una tupla de múltiples índices. Por ejemplo:

In [77]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1         2037.0
      2        12037.0
2014  1         2037.0
      2         -963.0
Name: (Bob, HR), dtype: float64

Trabajar con sectores dentro de estas tuplas de índice no es especialmente conveniente; intentar crear un segmento dentro de una tupla generará un error de sintaxis:

In [79]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (3311942670.py, line 1)

Puede evitar esto construyendo la porción deseada explícitamente usando el integrado de Python función ``slice()``, pero una mejor manera en este contexto es utilizar un objeto ``IndexSlice``, que Pandas proporciona precisamente para esta situación. Por ejemplo:

In [81]:
idx = pd.IndexSlice
health_data.loc[idx[:, 2], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,2,12037.0,-9963.0,-1963.0
2014,2,-963.0,-3963.0,15037.0


Hay tantas formas de interactuar con los datos en índices múltiples Series y DataFrames, y como con muchas herramientas en este libro, la mejor manera de familiarizarse con ellas es probarlas.

## Reordenando Multi-Índices

Una de las claves para trabajar con datos indexados de forma múltiple es saber cómo transformar los datos de manera efectiva. Hay una serie de operaciones que conservarán toda la información en el conjunto de datos, pero la reorganizarán para varios cálculos. Vimos un breve ejemplo de esto en los métodos ``stack()`` y ``unstack()``, pero hay muchas más formas de controlar finamente la reorganización de datos entre índices jerárquicos y columnas, y las exploraremos aquí.


### Índices ordenados y no ordenados

Mencionamos brevemente una advertencia, pero debemos enfatizarla más aquí. Mucho de las operaciones de ``MultiIndex`` fallarán si el índice no está ordenado. Echemos un vistazo a esto aquí.

Comenzaremos creando algunos datos indexados múltiples simples donde los índices no están ordenados lexográficamente :


In [82]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.085505
      2      0.854687
c     1      0.235770
      2      0.581263
b     1      0.940408
      2      0.900779
dtype: float64

Si intentamos tomar una porción parcial de este índice, dará como resultado un error:

In [83]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Aunque no está del todo claro en el mensaje de error, este es el resultado de que ``MultiIndex`` no está ordenado. Por varias razones, los cortes parciales y otras operaciones similares requieren los niveles en el MultiIndex estar en orden clasificado (es decir, lexográfico). Pandas ofrece una serie de prácticas rutinas para realizar este tipo de clasificación; ejemplos son los métodos ``sort_index()`` y ``sortlevel()`` de los ``DataFrames``. 

Usaremos el más simple, ``sort_index()``:

In [84]:
data = data.sort_index()
data

char  int
a     1      0.085505
      2      0.854687
b     1      0.940408
      2      0.900779
c     1      0.235770
      2      0.581263
dtype: float64

Con el índice ordenado de esta manera, el corte parcial funcionará como se esperaba:

In [85]:
data['a':'b']

char  int
a     1      0.085505
      2      0.854687
b     1      0.940408
      2      0.900779
dtype: float64

### Apilar y desapilar índices

Como vimos brevemente antes, es posible convertir un conjunto de datos de un índice múltiple apilado a una representación bidimensional simple, especificando opcionalmente el nivel a usar:

In [86]:
pop.unstack(level=0)

,California,New York,Texas
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [87]:
pop.unstack(level=1)

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Lo contrario a ``unstack()`` es ``stack()``, que aquí se puede utilizar para recuperar la serie original:

In [88]:
pop.unstack().stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Configuración y reinicio de índices

Otra forma de reorganizar los datos jerárquicos es convertir las etiquetas de índice en columnas; esto se puede lograr con el método ``reset_index``. Llamar esto en el diccionario de población resultará en un ``DataFrame`` con una estado y año que contiene la información que estaba anteriormente en el índice. Para mayor claridad, podemos especificar opcionalmente el nombre de los datos para la representación de la columna:

In [111]:
pop_flat = pop.reset_index(name='population')
pop_flat

,level_0,level_1,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


A menudo, cuando se trabaja con datos del mundo real, los datos de entrada sin procesar se ven así y son útiles para crear un ``MultiIndex`` de los valores de la columna. Esto se puede hacer con el metodo ``set_index`` de la ``DataFrame``, que devuelve un índice múltiple ``DataFrame``:

In [112]:
pop_flat=pop_flat.set_index(['level_0', 'level_1'])
pop_flat.index.names =['state','year']
pop_flat

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

En la práctica, encuentro que este tipo de reindexación es uno de los patrones más útiles cuando encuentro conjuntos de datos del mundo real.

## Agregaciones de datos en índices múltiples

Anteriormente vimos que Pandas tiene métodos integrados de agregación de datos, como ``mean()``, ``sum()``, y ``max()``. Para los datos indexados jerárquicamente, estos se pueden pasar un ``level`` parámetro que controla en qué subconjunto de los datos se calcula el agregado.

Por ejemplo, volvamos a nuestros datos de salud:

In [113]:
health_data

subject         Bob          Guido            Sue      
type             HR  Temp       HR  Temp       HR  Temp
year visit                                             
2013 1       2037.0  38.6   5037.0  37.2   7037.0  35.3
     2      12037.0  35.4  -9963.0  36.2  -1963.0  37.2
2014 1       2037.0  36.6 -23963.0  37.9  -3963.0  36.5
     2       -963.0  38.2  -3963.0  36.9  15037.0  38.1

Tal vez nos gustaría promediar las mediciones en las dos visitas cada año. Podemos hacer esto nombrando el nivel de índice que nos gustaría explorar, en este caso el año:

In [114]:
data_mean = health_data.mean(level='year')
data_mean

/tmp/ipykernel_4566/2845460446.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  data_mean = health_data.mean(level='year')


subject     Bob          Guido           Sue       
type         HR  Temp       HR  Temp      HR   Temp
year                                               
2013     7037.0  37.0  -2463.0  36.7  2537.0  36.25
2014      537.0  37.4 -13963.0  37.4  5537.0  37.30

Al seguir haciendo uso de la palabra clave ``axis``, también podemos tomar la media entre niveles en las columnas:

In [115]:
data_mean.mean(axis=1, level='type')

/tmp/ipykernel_4566/1641729190.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  data_mean.mean(axis=1, level='type')


type,HR,Temp
year,,
2013,2370.333333,36.650000
2014,-2629.666667,37.366667


Así, en dos líneas, hemos podido encontrar la frecuencia cardíaca y la temperatura promedio medidas entre todos los sujetos en todas las visitas cada año. Esta sintaxis es en realidad un atajo a la funcionalidad ``GroupBy``, de la que hablaremos en Agregación y agrupación . Si bien este es un ejemplo de juguete, muchos conjuntos de datos del mundo real tienen una estructura jerárquica similar.